# Lipschitz Constants and Automatic Step Size Tuning

## Introduction to Lipschitz Constants

In mathematical terms, a function $f: \mathbb{R}^n \rightarrow \mathbb{R}^m$ is said to be *Lipschitz continuous* if
there exists a constant $L$ such that for every pair of points $x, y \in \mathbb{R}^n$, the following inequality holds:

$$
\| f(x) - f(y) \| \leq L \| x - y \|.
$$

This equation essentially states that the change in the function's output cannot be faster than $L$ times the change in
the input. In simpler terms, a Lipschitz continuous function doesn't change too abruptly; it has a 'speed limit' defined
by its Lipschitz constant ($L$). For more in-depth knowledge, check the [Wikipedia page on Lipschitz
Continuity](https://en.wikipedia.org/wiki/Lipschitz_continuity).

Lipschitz constants are valuable tools in the realm of optimization, particularly for gradient-based methods like
gradient descent. Knowing the Lipschitz constant can help you set an effective step size, thereby ensuring stable and
faster convergence. However, computing these constants can be challenging. 

Good news! Pyxu offers unique features to automatically compute or estimate Lipschitz constants, making your life
easier. Let's dive into the details. 🌟

## Accessing Lipschitz Constants

Pyxu operators come equipped with `lipschitz`[🔗](../api/abc.html#pyxu.abc.Map.lipschitz) and
`diff_lipschitz`[🔗](../api/abc.html#pyxu.abc.DiffMap.diff_lipschitz) attributes which stores the Lipschitz constants of
maps and their derivatives (if defined), respectively. These constants are leveraged under the hood by Pyxu to auto-tune
the step sizes in various optimization algorithms.

```python
# Access Lipschitz constant of an operator 'op'
L = op.lipschitz
```

## Estimating Lipschitz Constants

For user-defined or complicated operators where Lipschitz constants are unknown, you can estimate them using the
`estimate_lipschitz()`[🔗](../api/abc.html#pyxu.abc.Map.estimate_lipschitz) method.

```python
# Estimate Lipschitz constant and update the attribute
L = op.estimate_lipschitz()
op.lipschitz = L
```

### Supported Backends 🎛️

Some operators offer several ways to estimate Lipschitz constants. When operators support this, their respective
`estimate_lipschitz()` or `estimate_diff_lipschitz()` methods document any extra parameters they may accept.
`LinOp()`[🔗](../api/abc.html#pyxu.abc.LinOp) in particular offers several methods to estimate its Lipschitz constants,
among which:

1. **Trace Method (**`trace`**)**: This is the default and computationally lighter option. It computes a rough estimate
   using the Frobenius norm of the operator, making use of the [Hutch++ stochastic algorithm](https://arxiv.org/abs/2010.09649):

   ```python
   # Using trace method
   op.lipschitz = op.estimate_lipschitz(method="trace")
   ```

2. **SVD Method (**`svd`**)**: This method computes the spectral norm of the operator and generally provides a tighter
   Lipschitz constant. However, it can be computationally intensive for large operators. A reduced-accuracy mode is
   available for quicker (but slightly overestimated) constants:

   ```python
   # Using SVD method with reduced accuracy
   op.lipschitz = op.estimate_lipschitz(method="svd", tol=1e-3)
   ```

> **Note 📝**: The Frobenius and spectral norms of $A: \mathbb{R}^{M} \to \mathbb{R}^{N}$ are related by $\|A\|_2\leq
> \|A\|_F\leq \sqrt{\min(N,M)} \|A\|_2$.

### Hands-On Example 🎓

Here is a practical example:

In [1]:
from pyxu.abc import LinOp
import numpy as np

rand_op = LinOp.from_array(np.random.random((10000, 10000)))

In [2]:
rand_op.lipschitz  # Unknown as this stage

inf

In [3]:
%%timeit
rand_op.lipschitz = rand_op.estimate_lipschitz(method="trace")

1.12 s ± 91.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [4]:
rand_op.lipschitz  # Rough estimate

5773.825067429628

In [5]:
%%timeit
rand_op.lipschitz = rand_op.estimate_lipschitz(method="svd", tol=1e-2)

1.38 s ± 220 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [6]:
rand_op.lipschitz # Tighter estimate

4999.816657444895

## Operator Algebra and Lipschitz Constant Propagation

Whenever possible, Lipschitz constants are propagated automatically by Pyxu's operator algebra logic.  More details on
this feature can be found in the [API reference](../api/index.html) under
`pyxu.abc.arithmetic`[🔗](../api/abc/arithmetic.html).

> **Note 📝**: While the propagated constants are usually good enough for step size tuning, they may not always be the
> tightest estimates.  You can always call `estimate_lipschitz()` again on arithmetic-produced operators to
> force-compute a tighter Lipschitz constant if needed.

```
op = op1 * op2  # arithmetic-induced operator
op.lipschitz  # => 50 (example value from cheap Lipschitz propagation.)
op.lipschitz = op.estimate_lipschitz()  # => 3 (re-compute a Lipschitz constant.)
```

And there you have it! With Pyxu, you're well-equipped to handle Lipschitz constants effectively, setting you on a
smooth path towards optimization success. 🚀